In [1]:
import os

In [2]:
# check current working directory
os.getcwd() 

'C:\\Users\\fanfan\\Documents\\Capstone\\DSCI_591_capstone-BCStats\\notebooks'

In [3]:
# change the working directory to the root directory of the project
os.chdir('../')
os.getcwd()

'C:\\Users\\fanfan\\Documents\\Capstone\\DSCI_591_capstone-BCStats'

In [73]:
import pandas as pd
import numpy as np
#import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from skmultilearn.problem_transform import BinaryRelevance

In [7]:
# read in 2018 qualitative data
df_2018 = pd.read_csv("data/interim/train_2018-qualitative-data.csv")

df_2018 = df_2018[['2018 Comment']].join(df_2018.loc[:,'CPD':'OTH'])
df_2018 = df_2018.rename(columns = {'2018 Comment' : 'comment'})

categories = df_2018.loc[:,'CPD':'OTH'].columns.tolist()

In [42]:
categories

['CPD',
 'CB',
 'EWC',
 'Exec',
 'FWE',
 'SP',
 'RE',
 'Sup',
 'SW',
 'TEPE',
 'VMG',
 'OTH']

In [9]:
df_2018.head()

,comment,CPD,CB,EWC,Exec,FWE,SP,RE,Sup,SW,TEPE,VMG,OTH
0,I would suggest having a developmental growth ...,1,0,0,0,0,1,0,0,0,0,0,0
1,Base decisions regarding fish and wildlife on ...,0,0,0,0,0,0,0,0,0,0,1,0
2,Get rid of Leading Workplace Strategies and gi...,0,0,0,0,1,0,0,0,0,0,0,0
3,We are the lowest paid in Canada with a worklo...,0,1,0,0,0,1,0,0,0,0,0,0
4,Official acknowledgement of the limited divers...,0,0,1,0,0,1,0,0,0,0,0,0


In [8]:
from src.data.preprocessing_text import (
    clean_text, clean_numbers, replace_typical_misspell, remove_stopwords,
    balance_themes
)

In [56]:
from src.features.word_vectors import (
    build_vocab, check_coverage, get_average_embeddings
)

In [76]:
from src.models.eval import theme_results

###  GloVe embeddings for Twitter

In [12]:
from gensim.models import KeyedVectors
glove_twitter_model = KeyedVectors.load_word2vec_format('references/pretrained_embeddings/glove.twitter.27B/glove.twitter.27B.200d.w2v.txt', binary=False)  # C text format

In [13]:
# get the vacabulary size of the glove_twitter_model
print("the vocabulary size in FastText model is",len(glove_twitter_model.wv.vocab))

the vocabulary size in FastText model is 1193514


C:\Users\fanfan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [29]:
# Preprocessing Data -- Remove punctuation, clean numbers, and fix spelling
df_2018["comment"] = df_2018["comment"].apply(lambda x: clean_text(x))

df_2018["comment"] = df_2018["comment"].apply(lambda x: clean_numbers(x))

df_2018["comment"] = df_2018["comment"].apply(
    lambda x: replace_typical_misspell(x)
)

sentences = df_2018["comment"].str.split()
sentences = remove_stopwords(sentences)

In [ ]:
#sentences

In [65]:
# Build average word vectors to train on
X_wv_twitter = np.array([get_average_embeddings(sentence, embeddings_index=glove_twitter_model, n_features=200)
                 for sentence in sentences])
Y_twitter = np.array(df_2018.loc[:,"CPD":"OTH"])

X_train_twitter, X_valid_twitter, Y_train_twitter, Y_valid_twitter = train_test_split(X_wv_twitter, Y_twitter, 
                                                            test_size=0.25, 
                                                            random_state=2019)

In [67]:
X_train_twitter.shape

(9958, 200)

In [68]:
X_valid_twitter.shape

(3320, 200)

In [69]:
Y_train_twitter.shape

(9958, 12)

In [70]:
Y_valid_twitter.shape

(3320, 12)

#### LinearSVC

In [75]:
classifier = BinaryRelevance(classifier=LinearSVC())
classifier.fit(X_train_twitter,Y_train_twitter)
prediction = classifier.predict(X_valid_twitter).toarray()

C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\fanfan\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iteration

In [77]:
theme_results(Y_valid_twitter,prediction)

Overall Accuracy: 0.3178 
Hamming Loss: 0.0913 
Hamming Loss (pred. zeros): 0.1191


,Label,Y_proportion,Pred_proportion,Error,Dummy_Diff,Accuarcy,Precision,Recall
0,CPD,0.127410,0.056024,0.099096,0.028313,0.900904,0.752688,0.330969
1,CB,0.184639,0.126807,0.089157,0.095482,0.910843,0.876485,0.601958
2,EWC,0.084337,0.013554,0.079217,0.005120,0.920783,0.688889,0.110714
3,Exec,0.103012,0.017470,0.098795,0.004217,0.901205,0.620690,0.105263
4,FWE,0.062048,0.025602,0.047892,0.014157,0.952108,0.776471,0.320388
5,SP,0.096386,0.025301,0.087952,0.008434,0.912048,0.666667,0.175000
6,RE,0.085542,0.008735,0.087048,-0.001506,0.912952,0.413793,0.042254
7,Sup,0.127711,0.029217,0.119578,0.008133,0.880422,0.639175,0.146226
8,SW,0.165964,0.060843,0.149096,0.016867,0.850904,0.638614,0.234120
9,TEPE,0.228614,0.193373,0.091265,0.137349,0.908735,0.855140,0.723320
